In [1]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
(PVDF-Al界面構造対応 修正版)

主な修正点:
1. 入力ファイル形式をCIFに変更
2. PVDF-Al系のファイル命名規則に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
5. 圧着ステップ(Step1)を削除
"""
from ase.constraints import FixAtoms
import numpy as np
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
import re

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# =============================================

# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure","output","PVDF_interfaces")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDF")
# OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDFedit")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile01")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# =============================================
# 統合システム対応のファイルパス関数
# =============================================

def get_interface_files():
    """統合システムで生成された界面構造ファイルを取得"""
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.cif')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出 (PVDF-Al系)
            if "PVDF_press_" in cif_file and "_on_" in cif_file:
                interface_files.append(cif_file)
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """
    界面構造ファイル名から材料情報を解析 (PVDF-Al系に対応)
    ファイル名例: 
    - "PVDF_press_on_Al_metal_final.cif"
    - "PVDF_press_on_Al2O3_final.cif"
    - "PVDF_press_on_AlF3_final.cif"
    """
    if not filename.startswith("PVDF_press_on_"):
        print(f"⏩ スキップ: ファイル名が 'PVDF_press_on_' で始まらないため処理対象外です: {filename}")
        return None
    
    base_name_for_error = Path(filename).stem
    
    try:
        cleaned_name = re.sub(r'\.cif$', '', filename)
        
        parts = cleaned_name.split('_on_')
        pvdf_part_str = parts[0] # "PVDF_press" (上部)
        al_part_str = parts[1]   # "Al_metal_final", "Al2O3_final" (基板)

        # --- Al側（基板）の解析 ---
        al_parts = al_part_str.split('_')
        if len(al_parts) > 1 and al_parts[-1] == "final":
            miller_al = al_parts.pop(-1) # "final" を便宜的にmiller指数として扱う
            al_component = "_".join(al_parts) # "Al_metal", "Al2O3", "AlF3"
        else:
            al_component = al_part_str
            miller_al = "unknown"
            
        # --- PVDF側（上部）の解析 ---
        nmc_component = "PVDF" # 便宜的にnmc_component変数名を利用
        nmc_variant = "unknown"
        miller_nmc = "unknown"
        
        if pvdf_part_str == "PVDF_press":
            nmc_component = "PVDF"
            nmc_variant = "press"
            miller_nmc = "bulk"
        if pvdf_part_str == "PVDF_press":
            nmc_component = "PVDF"
            nmc_variant = "press"
            miller_nmc = "bulk"
        else:
            nmc_component = pvdf_part_str

        # base_name は {上部}_{基板} に変更
        base_name = f"{pvdf_part_str}_on_{al_component}"

    except (ValueError, IndexError) as e:
        print(f"⚠️ ファイル名解析警告 ({filename}): {e}")
        return {
            'al_component': "unknown", 'nmc_component': "unknown",
            'nmc_variant': "unknown", 'miller_al': "unknown",
            'miller_nmc': "unknown", 'base_name': f"{base_name_for_error}"
        }

    return {
        'al_component': al_component,    # 基板: "Al_metal", "Al2O3", "AlF3"
        'nmc_component': nmc_component,  # 上部: "PVDF"
        'nmc_variant': nmc_variant,      # 上部詳細: "press"
        'miller_al': miller_al,          # 基板詳細: "final"
        'miller_nmc': miller_nmc,        # 上部詳細: "bulk"
        'base_name': base_name           # "PVDF_press_on_Al_metal" 等
    }


def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

# 圧着ステップ削除に伴い、get_compression_file_path は削除

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}__P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,           # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,             # 界面構造用vacuum（Å）
    "temperature": 300.0,          # 引張温度固定（K）
    "max_strain": 1.75,            # 界面用最大ひずみ
    "timestep": 1.0,               # fs
    "elastic_region": (0.005, 0.03), # 界面用弾性領域
    "quality_threshold": 0.85,     # 界面用R²値閾値
    "max_steps": 10000,            # 界面用最大ステップ数
    "output_freq": 50,             # 出力頻度
    "free_region_thickness": 36.0  # 界面構造用固定層厚さ（Å）
}

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,         # 界面分離判定の隙間（Å）
    "max_strain_limit": 1.75,      # 最大ひずみ制限
    "min_stress_threshold": -0.5,  # 最小応力閾値（GPa）
    "separation_wait_steps": 300,  # 分離後の待機ステップ
    "stress_drop_ratio": 0.7       # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
# 圧着ステップは削除されたが、ファイル名命名規則のために変数は残す
compression_temperatures = [300]     # 圧着シミュレーション温度（K）
compression_pressures = [0.00]       # 圧着圧力（GPa）
high_temperatures = [300]            # 高温シミュレーション温度（K）
tensile_temperature = 300.0          # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定 (PVDF-Al系に対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とPVDF原子（C, H, F）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    pvdf_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['C', 'H', ]]
    
    if not al_indices or not pvdf_indices:
        # 界面でない場合のフォールバック
        print("警告: identify_interface_region でAlまたはPVDF原子が見つかりません。")
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'pvdf_region': (np.mean(z_coords), np.max(z_coords)), # 'nmc_region' -> 'pvdf_region'
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    pvdf_z_coords = z_coords[pvdf_indices]
    
    # Al領域とPVDF領域の境界を特定
    al_center = np.mean(al_z_coords)
    pvdf_center = np.mean(pvdf_z_coords)
    
    if al_center < pvdf_center:
        # Alが下、PVDFが上
        interface_z = (np.max(al_z_coords) + np.min(pvdf_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    else:
        # PVDFが下、Alが上
        print("逆向きです。 (PVDFが下, Alが上)")
        interface_z = (np.max(pvdf_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'pvdf_region': pvdf_region,    # 'nmc_region' -> 'pvdf_region'
        'al_center': al_center,
        'pvdf_center': pvdf_center,  # 'nmc_center' -> 'pvdf_center'
        'is_interface': True
    }

def set_fixed_atoms_under(atoms,thickness=3.0):
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    indices = [i for i, z in enumerate(z_coords)if z < 3.0]
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    return atoms

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定 (PVDF-Al系対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得 (この関数は PVDF-Al 対応版)
    interface_info = identify_interface_region(atoms) 
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å")
        print(f"  PVDF領域: {interface_info['pvdf_region'][0]:.2f} - {interface_info['pvdf_region'][1]:.2f}Å") # NMC -> PVDF
        
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)
        free_z_max = min(free_z_max, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 構造情報を属性として保存（固定原子がなくても厚さ情報は保存）
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)

    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        atoms.info['fixed_atoms_count'] = 0
        atoms.info['free_atoms_count'] = len(atoms)
        # 既存の制約がクリアされていることを確認
        atoms.set_constraint() 
        return atoms

    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms


# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器 (PVDF-Al系対応)"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_pvdf_separation = False # al_nmc_separation -> al_pvdf_separation
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出 (PVDF-Al系対応)"""
        
        # Al-PVDF界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms) # PVDF-Al対応版
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region']
                pvdf_region = interface_info['pvdf_region'] # nmc_region -> pvdf_region
                
                # Al-PVDF間の隙間を計算
                if interface_info['al_center'] < interface_info['pvdf_center']: # nmc_center -> pvdf_center
                    gap = pvdf_region[0] - al_region[1] # nmc_region -> pvdf_region
                else:
                    gap = al_region[0] - pvdf_region[1] # nmc_region -> pvdf_region
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_pvdf_separation:
                    self.al_pvdf_separation = True # al_nmc_separation -> al_pvdf_separation
                    self.debonding_step = system.current_total_step
                    print(f"        🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] / units.GPa
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15: 
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"        ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_pvdf_separation: # al_nmc_separation -> al_pvdf_separation
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す (PVDF-Al系対応)"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_pvdf_separation, # al_nmc_separation -> al_pvdf_separation
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理 (PVDF-Al系対応)"""
    
    print("=" * 80)
    print("PVDF-Al 界面構造MD解析システム")
    print("統合界面構造生成システムとの連携版")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    original_count = len(interface_files)
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        return
    # original_count = len(interface_files)
    interface_files = [
        f for f in interface_files 
        if not Path(f).stem.endswith("_pre_opt")
    ]
    print(f"解析対象の界面構造: {len(interface_files)}個")
    removed_count = original_count - len(interface_files)
    processed_count = 0
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            # if material_info is None:
            #     continue # スキップ
            
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"材料: {material_info}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # ファイルパス設定
                            # 圧着ステップは削除
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)

                            # ステップ1: 圧着 (削除)
                            # 圧着ステップは実行しない

                            # ステップ2: 平衡化 & 高温処理
                            if not check_md_completed(high_temp_file, 10000, 100):
                                print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                
                                # 圧着後ファイル(compression_file)の代わりに、
                                # 直接入力ファイル(input_file)を読む
                                atoms_after_comp = read(input_file)
                                pos=atoms_after_comp.get_positions()
                                indices_to_delete1 = np.where((pos[:, 2] >= 100))
                                del atoms_after_comp[indices_to_delete1]
                                # atoms_after_comp
                                # set_fixed_atoms_under(atoms_after_comp) # 下部を固定
                                positions = atoms_after_comp.get_positions()
                                z_coords = positions[:, 2]
                                indices = [i for i, z in enumerate(z_coords)if z > 35 or z<5]
                                # 固定制約の設定
                                if len(indices) == 0:
                                    print("  警告: 固定原子がありません")
                                    return atoms
                                # constraint = FixAtoms(indices=indices)
                                # atoms_after_comp.set_constraint(constraint)
                                # atoms_after_comp = set_interface_fixed_atoms_uniform_free(atoms_after_comp, UNIFIED_CONDITIONS["free_region_thickness"])
                                
                                # 平衡化 (NVT, 3ps @ 300K)
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(300.0)
                                integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                                
                                # 高温処理 (NVT, 10ps @ high_temp)
                                atoms_after_equil = read(equil_file, index='-1')
                                # set_fixed_atoms_under(atoms_after_equil) # 高温処理では固定層を再設定しない？
                                positions = atoms_after_equil.get_positions()
                                z_coords = positions[:, 2]
                                indices = [i for i, z in enumerate(z_coords)if z > 35 or z<5]
                                # 固定制約の設定
                                if len(indices) == 0:
                                    print("  警告: 固定原子がありません")
                                    return atoms
                                constraint = FixAtoms(indices=indices)
                                atoms_after_equil.set_constraint(constraint)
                                atoms_after_equil = set_interface_fixed_atoms_uniform_free(atoms_after_equil, UNIFIED_CONDITIONS["free_region_thickness"])
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(float(high_temp))
                                integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                            else:
                                print("高温シミュレーションは既に完了しています。")

                            # ステップ3: 引張試験
                            if not os.path.exists(summary_file):
                                print("引張シミュレーション実行中...")
                                final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                
                                print(f"\n✅ 引張計算完了: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                            else:
                                print("引張シミュレーションと解析は既に完了しています。")

                        except Exception as e:
                            print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                            import traceback
                            traceback.print_exc()
                            continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        # ステップ数が期待値の90%以上あれば完了とみなす
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """
    界面構造ファイルを読み込み、MD計算用に準備する関数。
    (PVDF-Al系対応 / 修正済みバージョン)
    """
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # --- 1. 構造の読み込みと基本設定 ---
    atoms = Trajectory(input_file)[-1]
    atoms.set_constraint() # 既存の拘束をクリア

    #--- 2. 構造の向きを調整 (Alが下層になるように条件付きで反転) ---
    al_indices  = [a.index for a in atoms if a.symbol == 'Al']
    # PVDF (C, H, F) を 'other' とみなす
    other_indices = [a.index for a in atoms if a.symbol in ['C', 'H', 'F']]

    if al_indices and other_indices:
        al_z_mean = np.mean(atoms.get_positions()[al_indices, 2])
        other_z_mean = np.mean(atoms.get_positions()[other_indices, 2])

        if al_z_mean > other_z_mean:
            print("構造チェック: Al層が上部に検出されたため、構造をZ軸反転します。")
            positions = atoms.get_positions()
            positions[:, 2] *= -1
            atoms.set_positions(positions)
        else:
            print("構造チェック: Al層は既に下部にあるため、構造の反転はスキップします。")
    else:
        print("構造チェック: Al層またはPVDF層が検出できなかったため、反転処理をスキップします。")

    # --- 3. 構造情報の解析とz_minの0への移動 ---
    positions = atoms.get_positions()
    z_min = np.min(positions[:, 2])
    
    atoms.translate([0, 0, -z_min])
    print(f"構造移動: z_min ({z_min:.2f} Å) が 0 Å になるように構造全体を移動しました。")

    z_coords = atoms.get_positions()[:, 2]
    z_min_new, z_max_new = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max_new - z_min_new
    
    print("界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f} Å")
    print(f"  原子数: {len(atoms)}")

    # --- 4. セルと真空層の設定 ---
    # vacuum_top 引数を尊重せず、1.0 Å を追加する (元のコードの挙動を維持)
    vacuum_to_add = 1.0 
    print(f"設定: 構造上部に {vacuum_to_add} Å の真空層を追加します。")
    
    new_cell_z = material_thickness + vacuum_to_add
    cell = atoms.get_cell()
    cell[2, 2] = new_cell_z
    atoms.set_cell(cell, scale_atoms=False) 

    # ひずみ基準長
    strain_reference_length = material_thickness
    strain_method = "material_length"

    # --- 5. 自由領域・固定領域の設定 ---
    # この時点で atoms.info に 'free_region_thickness' などが設定される
    atoms = set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness) 

    # --- 6. 構造情報を `atoms.info` に集約 ---
    interface_info = identify_interface_region(atoms) # 情報を再取得
    initial_reference = atoms.get_cell()[2, 2]
    total_cell_z = atoms.get_cell()[2, 2]

    # atoms.info を更新 (set_interface_fixed... で追加された情報以外を update)
    atoms.info.update({
        "input_file": os.path.basename(input_file),
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": total_cell_z,
        "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
        "strain_reference_length": strain_reference_length,
        "strain_method": strain_method,
        "interface_info": interface_info,
        # 'free_region_thickness' などは set_interface... 関数内で既に atoms.info に追加されている
    })
    
    # 呼び出し元に返す structure_info 辞書を作成
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
        "interface_info": interface_info,
        "strain_method": strain_method,
        
        # set_interface_fixed... で設定された情報を atoms.info から取得
        "free_region_thickness": atoms.info.get('free_region_thickness', None),
        "free_region_bounds": atoms.info.get('free_region_bounds', None),
        "fixed_atoms_count": atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": atoms.info.get('free_atoms_count', len(atoms))
    }
    
    print("準備完了:")
    print(f"  最終セルZ長: {atoms.info['total_cell_z']:.2f} Å")
    print(f"  材料占有率: {atoms.info['material_ratio']*100:.1f}%")
    if atoms.info['interface_info'].get('is_interface', False):
        print(f"  界面検出: Yes (Z ≈ {atoms.info['interface_info'].get('interface_z', 0.0):.2f} Å)")
    else:
        print(f"  界面検出: No")
    
    return atoms, initial_reference, structure_info
    

# 圧着ステップ削除に伴い、run_interface_compression_md は削除

def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    ase_atoms,initial_reference,structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if initial_reference == 0:
        initial_reference = 50 # Fallback value to avoid division by zero
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")

    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版 / PVDF-Al系対応）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"PVDF-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  基板成分 (Al側): {material_info['al_component']}\n")
        f.write(f"  上部成分 (PVDF側): {material_info['nmc_component']}\n") # nmc_component 変数名だが中身は PVDF
        f.write(f"  上部組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  基板 Miller指数: {material_info['miller_al']}\n")
        f.write(f"  上部 Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        free_count = structure_info.get('free_atoms_count', 0)
        fixed_count = structure_info.get('fixed_atoms_count', 1) # Avoid division by zero
        total_atoms = free_count + fixed_count
        free_ratio = free_count / total_atoms if total_atoms > 0 else 0

        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {free_count}\n")
        f.write(f"  固定原子数: {fixed_count}\n")
        f.write(f"  自由原子比率: {free_ratio * 100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力 (ファイル名用): {pressure:.3f} GPa\n")
        f.write(f"  温度 (ファイル名用): {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
            
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")


if __name__ == "__main__":
    main()

PVDF-Al 界面構造MD解析システム
統合界面構造生成システムとの連携版
発見された界面構造ファイル数: 0
❌ 界面構造ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces_onlyH2O


In [ ]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
(PVDF-Al界面構造対応 修正版 V2)

主な修正点:
1. 入力ファイル形式をCIFに変更
2. PVDF-Al系のファイル命名規則に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
5. 圧着ステップ(Step1)を削除
6. ★ 2_build_interfaces_v2.py の新しいファイル命名規則
   (_repeat_..., _cropped_... など) に対応
"""

from ase.io import read, Trajectory, write
# ... (他の import は変更なし) ...
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
from ase.constraints import FixAtoms
import numpy as np
import re
import os

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# (変更なし)
# =============================================
# ... (BASE_DIR, INPUT_DIR などの定義) ...
# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure","output","PVDF_2x2x1")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDF")
# OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDFedit")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile01")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)


# =============================================
# 統合システム対応のファイルパス関数 (修正あり)
# =============================================

def get_interface_files():
    """
    統合システムで生成された界面構造ファイルを取得
    ★ 新旧の命名規則 (PVDF_on_... と PVDF_press_on_...) の両方を対象
    """
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.xyz')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出 (PVDF-Al系)
            # ★ 新しい形式 'PVDF_on_' と 古い形式 'PVDF_press_on_' の両方を許可
            if "PVDF_on_" in cif_file or "PVDF_press_on_" in cif_file:
                interface_files.append(cif_file)
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """
    界面構造ファイル名から材料情報を解析 (PVDF-Al系に対応)
    ★ 新しい命名規則（_repeat_... や _cropped_...）に対応 (V2)

    ファイル名例:
    - "PVDF_on_Al_metal_repeat_2x2x1_final.cif"
    - "PVDF_on_Al2O3_cropped_size_15x15_final.cif"
    - "PVDF_on_AlF3_original_cell_final.cif"
    - "PVDF_press_on_AlF3_final.cif" (古い形式もサポート)
    """
    
    # .cif を削除
    base_name_for_error = Path(filename).stem # "PVDF_on_Al_metal_repeat_2x2x1_final"
    
    # --- 1. "_final" を削除 ---
    if base_name_for_error.endswith("_final"):
        base_name_no_final = base_name_for_error[:-len("_final")]
    elif base_name_for_error.endswith("_optimization"):
        base_name_no_final = base_name_for_error[:-len("_optimization")]
    # elif base_name_for_error.endswith("_pre_opt"):
        # base_name_no_final = base_name_for_error[:-len("_optimization")]
    else:
        # 古い形式 (PVDF_press_on_...) は _final が無い場合もあるため、
        # "PVDF_press_on_" で始まらない場合は _final が必須だったとみなし、
        # 解析エラーになるよう "unknown" を返す
        if not base_name_for_error.startswith("PVDF_press_on_"):
             print(f"⏩ スキップ: ファイル名に '_final' が含まれていません: {filename}")
             return None
        
        # 古い形式 "PVDF_press_on_..." の場合は _final がなくても解析を試みる
        base_name_no_final = base_name_for_error

    # --- 2. 新しいサフィックス (_repeat_, _cropped_, _original_) を特定・分離 ---
    # (例: "PVDF_on_Al_metal_repeat_2x2x1")
    
    substrate_type_suffix = "unknown" # "repeat_2x2x1", "cropped_size_15x15", "original_cell" など
    core_name = base_name_no_final # "PVDF_on_Al_metal_repeat_2x2x1"
    
    # 正規表現でサフィックスを検索
    # (末尾の _repeat_... or _cropped_abs_... or _cropped_size_... or _original_cell)
    # (注: _cropped_abs_... は Lx_str が "20_5" のように "_" を含む可能性があるため、
    #    単純な \w+ ではなく .* (貪欲でない) を使う必要があるが、
    #    ここでは末尾までをサフィックスとみなすため、パターンを限定する)
    pattern = r"^(.*?)(_repeat_\d+x\d+x\d+|_cropped_abs_[\d_x]+|_cropped_size_[\d_x]+|_original_cell)$"
    match = re.match(pattern, base_name_no_final)
    
    if match:
        core_name = match.group(1) # "PVDF_on_Al_metal"
        substrate_type_suffix = match.group(2)[1:] # "_repeat_..." -> "repeat_..."
    else:
        # マッチしない場合 (古い形式 "PVDF_press_on_Al_metal" など)
        core_name = base_name_no_final
        substrate_type_suffix = "legacy_format" # 古い形式

    # --- 3. コア部分 ("PVDF_on_Al_metal" or "PVDF_press_on_Al_metal") を解析 ---
    
    # "PVDF_press_on_" (古い形式) または "PVDF_on_" (新しい形式) で分割
    parts = []
    if "_on_" in core_name:
        # 貪欲に分割 (例: "PVDF_press_on_Al_metal")
        parts = core_name.split('_on_', 1) 
    
    if len(parts) != 2:
        # 古い形式 "PVDF_press_on_..." を再チェック
        if core_name.startswith("PVDF_press_on_"):
            pvdf_part_str = "PVDF_press"
            al_part_str = core_name[len("PVDF_press_on_"):]
            if substrate_type_suffix == "legacy_format":
                substrate_type_suffix = "legacy_press_format" # 古い形式であることを明記
        else:
            print(f"⏩ スキップ: ファイル名のコア部分 '{core_name}' (from {filename}) が '_on_' で分割できません。")
            return None
    else:
        pvdf_part_str = parts[0] # "PVDF" or "PVDF_press"
        al_part_str = parts[1]   # "Al_metal", "Al2O3", "AlF3"
    
    # --- 4. 各コンポーネントを整理 ---
    
    # Al側（基板）の解析
    al_component = al_part_str # "Al_metal", "Al2O3", "AlF3"
    
    # PVDF側（上部）の解析
    nmc_component = "PVDF" # 便宜的にnmc_component変数名を利用
    nmc_variant = "unknown"
    
    if pvdf_part_str == "PVDF":
        nmc_component = "PVDF"
        nmc_variant = "standard" # 新しい形式
    elif pvdf_part_str == "PVDF_press":
        nmc_component = "PVDF"
        nmc_variant = "press" # 古い形式
    else:
        # 万が一 "Something_else_on_Al_metal" だった場合
        nmc_component = pvdf_part_str
        nmc_variant = "custom_top"
        
    # miller_al の代わりに substrate_type_suffix を使用
    # miller_nmc は "unknown" または "bulk" (古い形式の場合)
    miller_nmc = "unknown"
    if nmc_variant == "press":
        miller_nmc = "bulk"

    # base_name は {上部}_{基板}_{サフィックス} に変更
    # (例: "PVDF_on_Al_metal_repeat_2x2x1")
    # ただし、サフィックスが "legacy_format" や "legacy_press_format" の場合は追加しない
    base_name = f"{pvdf_part_str}_on_{al_component}"
    if substrate_type_suffix not in ["legacy_format", "legacy_press_format", "unknown"]:
        base_name += f"_{substrate_type_suffix}"
    
    # 古い形式 "PVDF_press_on_Al_metal" の場合、
    # substrate_type_suffix は "legacy_press_format" になり、
    # base_name は "PVDF_press_on_Al_metal" となる。
    
    # 新しい形式 "PVDF_on_Al_metal_repeat_2x2x1" の場合、
    # substrate_type_suffix は "repeat_2x2x1" になり、
    # base_name は "PVDF_on_Al_metal_repeat_2x2x1" となる。

    return {
        'al_component': al_component,      # 基板: "Al_metal", "Al2O3", "AlF3"
        'nmc_component': nmc_component,  # 上部: "PVDF"
        'nmc_variant': nmc_variant,      # 上部詳細: "standard" or "press"
        'miller_al': substrate_type_suffix, # 基板詳細: "repeat_2x2x1", "cropped_...", "original_cell", "legacy_..."
        'miller_nmc': miller_nmc,        # 上部詳細: "unknown" or "bulk"
        'base_name': base_name           # "PVDF_on_Al_metal_repeat_2x2x1" 等
    }

# ... (get_input_file_path から save_interface_results までの関数は変更なし) ...
def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

# 圧着ステップ削除に伴い、get_compression_file_path は削除

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}__P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,         # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,           # 界面構造用vacuum（Å）
    "temperature": 300.0,        # 引張温度固定（K）
    "max_strain": 1.75,          # 界面用最大ひずみ
    "timestep": 1.0,             # fs
    "elastic_region": (0.005, 0.03), # 界面用弾性領域
    "quality_threshold": 0.85,     # 界面用R²値閾値
    "max_steps": 10000,          # 界面用最大ステップ数
    "output_freq": 50,           # 出力頻度
    "free_region_thickness": 36.0  # 界面構造用固定層厚さ（Å）
}

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,         # 界面分離判定の隙間（Å）
    "max_strain_limit": 1.75,      # 最大ひずみ制限
    "min_stress_threshold": -0.5,  # 最小応力閾値（GPa）
    "separation_wait_steps": 300,  # 分離後の待機ステップ
    "stress_drop_ratio": 0.7       # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
# 圧着ステップは削除されたが、ファイル名命名規則のために変数は残す
compression_temperatures = [300]     # 圧着シミュレーション温度（K）
compression_pressures = [0.00]       # 圧着圧力（GPa）
high_temperatures = [1600]            # 高温シミュレーション温度（K）
tensile_temperature = 300.0          # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定 (PVDF-Al系に対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とPVDF原子（C, H, F）を分離
    # ★ Al2O3, AlF3 にも対応するため、Al 以外の原子も Al 層とみなす必要がある
    #    → PVDF (C, H, F) 以外の全てを "基板" とみなす
    
    pvdf_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['C', 'H', 'F']]
    # ★ 基板 (Substrate) は PVDF 以外の全原子
    substrate_indices = [i for i, atom in enumerate(atoms) if atom.symbol not in ['C', 'H', 'F']]
    
    # (元の al_indices を substrate_indices に変更)
    if not substrate_indices or not pvdf_indices:
        # 界面でない場合のフォールバック
        print("警告: identify_interface_region で基板またはPVDF原子が見つかりません。")
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)), # 'al_region' 変数名は維持
            'pvdf_region': (np.mean(z_coords), np.max(z_coords)), # 'nmc_region' -> 'pvdf_region'
            'is_interface': False
        }
    
    substrate_z_coords = z_coords[substrate_indices]
    pvdf_z_coords = z_coords[pvdf_indices]
    
    # Al領域とPVDF領域の境界を特定
    substrate_center = np.mean(substrate_z_coords) # al_center -> substrate_center
    pvdf_center = np.mean(pvdf_z_coords)
    
    if substrate_center < pvdf_center:
        # 基板が下、PVDFが上
        interface_z = (np.max(substrate_z_coords) + np.min(pvdf_z_coords)) / 2
        al_region = (np.min(substrate_z_coords), np.max(substrate_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    else:
        # PVDFが下、基板が上
        print("逆向きです。 (PVDFが下, 基板が上)")
        interface_z = (np.max(pvdf_z_coords) + np.min(substrate_z_coords)) / 2
        al_region = (np.min(substrate_z_coords), np.max(substrate_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,     # 'al_region' 変数名は維持 (基板領域)
        'pvdf_region': pvdf_region,   # 'nmc_region' -> 'pvdf_region'
        'al_center': substrate_center, # 'al_center' 変数名は維持 (基板中心)
        'pvdf_center': pvdf_center,   # 'nmc_center' -> 'pvdf_center'
        'is_interface': True
    }

def set_fixed_atoms_under(atoms,thickness=3.0):
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    indices = [i for i, z in enumerate(z_coords)if z < 3.0]
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    return atoms

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定 (PVDF-Al系対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得 (この関数は PVDF-Al 対応版)
    interface_info = identify_interface_region(atoms) 
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  基板領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å") # Al -> 基板
        print(f"  PVDF領域: {interface_info['pvdf_region'][0]:.2f} - {interface_info['pvdf_region'][1]:.2f}Å") # NMC -> PVDF
        
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)
        free_z_max = min(free_z_max, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 構造情報を属性として保存（固定原子がなくても厚さ情報は保存）
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)

    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        atoms.info['fixed_atoms_count'] = 0
        atoms.info['free_atoms_count'] = len(atoms)
        # 既存の制約がクリアされていることを確認
        atoms.set_constraint() 
        return atoms

    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms


# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器 (PVDF-Al系対応)"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_pvdf_separation = False # al_nmc_separation -> al_pvdf_separation
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出 (PVDF-Al系対応)"""
        
        # Al-PVDF界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms) # PVDF-Al対応版
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region'] # 基板領域
                pvdf_region = interface_info['pvdf_region'] # nmc_region -> pvdf_region
                
                # Al-PVDF間の隙間を計算
                if interface_info['al_center'] < interface_info['pvdf_center']: # nmc_center -> pvdf_center
                    gap = pvdf_region[0] - al_region[1] # nmc_region -> pvdf_region
                else:
                    gap = al_region[0] - pvdf_region[1] # nmc_region -> pvdf_region
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_pvdf_separation:
                    self.al_pvdf_separation = True # al_nmc_separation -> al_pvdf_separation
                    self.debonding_step = system.current_total_step
                    print(f"         🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] / units.GPa
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15: 
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"         ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_pvdf_separation: # al_nmc_separation -> al_pvdf_separation
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す (PVDF-Al系対応)"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_pvdf_separation, # al_nmc_separation -> al_pvdf_separation
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理 (PVDF-Al系対応)"""
    
    print("=" * 80)
    print("PVDF-Al 界面構造MD解析システム")
    print("統合界面構造生成システムとの連携版 (V2)")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    processed_count = 0
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            original_count = len(interface_files)
            if not interface_files:
                print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
                return
            # original_count = len(interface_files)
            interface_files = [
                f for f in interface_files 
                if not Path(f).stem.endswith("_pre_opt")
            ]
            print(f"解析対象の界面構造: {len(interface_files)}個")
            removed_count = original_count - len(interface_files)
            if material_info is None:
                continue # スキップ
            
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"  材料: {material_info}")
            print(f"  BaseName: {base_name}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # ファイルパス設定
                            # 圧着ステップは削除
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)

                            # ステップ1: 圧着 (削除)
                            # 圧着ステップは実行しない

                            # ステップ2: 平衡化 & 高温処理
                            if not check_md_completed(high_temp_file, 10000, 100):
                                print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                
                                # 圧着後ファイル(compression_file)の代わりに、
                                # 直接入力ファイル(input_file)を読む
                                atoms_after_comp = read(input_file)
                                
                                # ★ 元のスクリプトにあった固定層のロジック (V1 でコメントアウトされていたもの)
                                #    ここでは V2 の固定層設定 (set_interface_fixed_atoms_uniform_free)
                                #    を prepare_interface_structure_for_md の中で行うため、
                                #    ここでの固定層設定は不要 (V1 のロジックも削除)
                                
                                # pos=atoms_after_comp.get_positions()
                                # indices_to_delete1 = np.where((pos[:, 2] >= 100))
                                # del atoms_after_comp[indices_to_delete1]
                                # positions = atoms_after_comp.get_positions()
                                # z_coords = positions[:, 2]
                                # indices = [i for i, z in enumerate(z_coords)if z > 35 or z<5]
                                # if len(indices) == 0:
                                #     print("  警告: 固定原子がありません")
                                #     # return atoms # main() の中なので return は不適切
                                # else:
                                #     constraint = FixAtoms(indices=indices)
                                #     atoms_after_comp.set_constraint(constraint)
                                
                                # ★ V2: set_interface_fixed_atoms_uniform_free を使う
                                #     (ただし、高温処理の前に再度設定する)
                                atoms_after_comp = set_interface_fixed_atoms_uniform_free(
                                    atoms_after_comp, 
                                    UNIFIED_CONDITIONS["free_region_thickness"]
                                )
                                
                                # 平衡化 (NVT, 3ps @ 300K)
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(300.0)
                                integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                                
                                # 高温処理 (NVT, 10ps @ high_temp)
                                atoms_after_equil = read(equil_file, index='-1')
                                
                                # ★ 高温処理の前に固定層を再設定
                                atoms_after_equil = set_interface_fixed_atoms_uniform_free(
                                    atoms_after_equil, 
                                    UNIFIED_CONDITIONS["free_region_thickness"]
                                )
                                
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(float(high_temp))
                                integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                            else:
                                print("高温シミュレーションは既に完了しています。")

                            # ステップ3: 引張試験
                            if not os.path.exists(summary_file):
                                print("引張シミュレーション実行中...")
                                final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                
                                print(f"\n✅ 引張計算完了: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                            else:
                                print("引張シミュレーションと解析は既に完了しています。")

                        except Exception as e:
                            print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                            import traceback
                            traceback.print_exc()
                            continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        # ステップ数が期待値の90%以上あれば完了とみなす
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """
    界面構造ファイルを読み込み、MD計算用に準備する関数。
    (PVDF-Al系対応 / 修正済みバージョン)
    """
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # --- 1. 構造の読み込みと基本設定 ---
    atoms = Trajectory(input_file)[-1]
    atoms.set_constraint() # 既存の拘束をクリア

    #--- 2. 構造の向きを調整 (基板が下層になるように条件付きで反転) ---
    # (★ identify_interface_region を使うように変更)
    interface_info_check = identify_interface_region(atoms)
    
    if interface_info_check['is_interface']:
        # al_center は "基板中心" を意味する
        if interface_info_check['al_center'] > interface_info_check['pvdf_center']:
            print("構造チェック: 基板層が上部に検出されたため、構造をZ軸反転します。")
            positions = atoms.get_positions()
            positions[:, 2] *= -1
            atoms.set_positions(positions)
        else:
            print("構造チェック: 基板層は既に下部にあるため、構造の反転はスキップします。")
    else:
        print("構造チェック: 界面が検出できなかったため、反転処理をスキップします。")

    # --- 3. 構造情報の解析とz_minの0への移動 ---
    positions = atoms.get_positions()
    z_min = np.min(positions[:, 2])
    
    atoms.translate([0, 0, -z_min])
    print(f"構造移動: z_min ({z_min:.2f} Å) が 0 Å になるように構造全体を移動しました。")

    z_coords = atoms.get_positions()[:, 2]
    z_min_new, z_max_new = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max_new - z_min_new
    
    print("界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f} Å")
    print(f"  原子数: {len(atoms)}")

    # --- 4. セルと真空層の設定 ---
    # vacuum_top 引数を尊重せず、1.0 Å を追加する (元のコードの挙動を維持)
    vacuum_to_add = 1.0 
    print(f"設定: 構造上部に {vacuum_to_add} Å の真空層を追加します。")
    
    new_cell_z = material_thickness + vacuum_to_add
    cell = atoms.get_cell()
    cell[2, 2] = new_cell_z
    atoms.set_cell(cell, scale_atoms=False) 

    # ひずみ基準長
    strain_reference_length = material_thickness
    strain_method = "material_length"

    # --- 5. 自由領域・固定領域の設定 ---
    # この時点で atoms.info に 'free_region_thickness' などが設定される
    atoms = set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness) 

    # --- 6. 構造情報を `atoms.info` に集約 ---
    interface_info = identify_interface_region(atoms) # 情報を再取得
    initial_reference = atoms.get_cell()[2, 2]
    total_cell_z = atoms.get_cell()[2, 2]

    # atoms.info を更新 (set_interface_fixed... で追加された情報以外を update)
    atoms.info.update({
        "input_file": os.path.basename(input_file),
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": total_cell_z,
        "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
        "strain_reference_length": strain_reference_length,
        "strain_method": strain_method,
        "interface_info": interface_info,
        # 'free_region_thickness' などは set_interface... 関数内で既に atoms.info に追加されている
    })
    
    # 呼び出し元に返す structure_info 辞書を作成
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
        "interface_info": interface_info,
        "strain_method": strain_method,
        
        # set_interface_fixed... で設定された情報を atoms.info から取得
        "free_region_thickness": atoms.info.get('free_region_thickness', None),
        "free_region_bounds": atoms.info.get('free_region_bounds', None),
        "fixed_atoms_count": atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": atoms.info.get('free_atoms_count', len(atoms))
    }
    
    print("準備完了:")
    print(f"  最終セルZ長: {atoms.info['total_cell_z']:.2f} Å")
    print(f"  材料占有率: {atoms.info['material_ratio']*100:.1f}%")
    if atoms.info['interface_info'].get('is_interface', False):
        print(f"  界面検出: Yes (Z ≈ {atoms.info['interface_info'].get('interface_z', 0.0):.2f} Å)")
    else:
        print(f"  界面検出: No")
    
    return atoms, initial_reference, structure_info
    

# 圧着ステップ削除に伴い、run_interface_compression_md は削除

def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    ase_atoms,initial_reference,structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if initial_reference == 0:
        initial_reference = 50 # Fallback value to avoid division by zero
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")

    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版 / PVDF-Al系対応）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"PVDF-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  基板成分 (Al側): {material_info['al_component']}\n")
        f.write(f"  上部成分 (PVDF側): {material_info['nmc_component']}\n") # nmc_component 変数名だが中身は PVDF
        f.write(f"  上部組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  基板 Miller指数/タイプ: {material_info['miller_al']}\n") # ★ 名前変更
        f.write(f"  上部 Miller指数/タイプ: {material_info['miller_nmc']}\n") # ★ 名前変更
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        free_count = structure_info.get('free_atoms_count', 0)
        fixed_count = structure_info.get('fixed_atoms_count', 1) # Avoid division by zero
        total_atoms = free_count + fixed_count
        free_ratio = free_count / total_atoms if total_atoms > 0 else 0

        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {free_count}\n")
        f.write(f"  固定原子数: {fixed_count}\n")
        f.write(f"  自由原子比率: {free_ratio * 100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力 (ファイル名用): {pressure:.3f} GPa\n")
        f.write(f"  温度 (ファイル名用): {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
            
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")


if __name__ == "__main__":
    main()

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_Al_cell_repeat_2x2x1_P0.000_T300K_equil.traj.
Note: The max disk size of /home/jovyan is about 98G.


PVDF-Al 界面構造MD解析システム
統合界面構造生成システムとの連携版 (V2)
発見された界面構造ファイル数: 3
解析対象の界面構造: 3個
解析対象の界面構造: 3個

処理開始: PVDF_on_Al_cell_repeat_2x2x1_final.xyz
  材料: {'al_component': 'Al_cell', 'nmc_component': 'PVDF', 'nmc_variant': 'standard', 'miller_al': 'repeat_2x2x1', 'miller_nmc': 'unknown', 'base_name': 'PVDF_on_Al_cell_repeat_2x2x1'}
  BaseName: PVDF_on_Al_cell_repeat_2x2x1

--- 条件: P0.000_T300K_HT1600K ---
平衡化と高温シミュレーション(1600K)を実行中...
自由領域統一固定層設定:
  総材料厚さ: 33.28Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 18.71Å
  基板領域: 0.00 - 18.77Å
  PVDF領域: 18.65 - 33.28Å
  自由領域: 2.00 - 31.28Å
  実際の自由領域厚さ: 29.28Å
  固定原子数: 279/1825 (15.3%)
  自由原子数: 1546/1825 (84.7%)


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_Al_cell_repeat_2x2x1_P0.000_T300K_HT1600K.traj.
Note: The max disk size of /home/jovyan is about 98G.


自由領域統一固定層設定:
  総材料厚さ: 33.28Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 20.62Å
  基板領域: 0.00 - 20.98Å
  PVDF領域: 20.26 - 33.28Å
  自由領域: 2.62 - 31.28Å
  実際の自由領域厚さ: 28.66Å
  固定原子数: 342/1825 (18.7%)
  自由原子数: 1483/1825 (81.3%)


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/tensile01/PVDF_on_Al_cell_repeat_2x2x1_P0.000_T300K_HT1600K_tensile300K.traj.
Note: The max disk size of /home/jovyan is about 98G.


引張シミュレーション実行中...
構造チェック: 基板層は既に下部にあるため、構造の反転はスキップします。
構造移動: z_min (0.00 Å) が 0 Å になるように構造全体を移動しました。
界面構造解析:
  材料厚さ: 33.28 Å
  原子数: 1825
設定: 構造上部に 1.0 Å の真空層を追加します。
自由領域統一固定層設定:
  総材料厚さ: 33.28Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 17.19Å
  基板領域: 0.00 - 27.55Å
  PVDF領域: 6.83 - 33.28Å
  自由領域: 2.00 - 31.28Å
  実際の自由領域厚さ: 29.28Å
  固定原子数: 297/1825 (16.3%)
  自由原子数: 1528/1825 (83.7%)
準備完了:
  最終セルZ長: 34.28 Å
  材料占有率: 97.1%
  界面検出: Yes (Z ≈ 17.19 Å)
自由領域統一引張設定:
  統一自由領域厚さ: 29.28Å
  自由原子数: 1528
  固定原子数: 297
  引張ひずみ速度: 0.01 1/ps
Dyn step    0 a 25.77 b 25.77 c 34.28
Dyn step 1000 a 25.77 b 25.77 c 40.27
Dyn step 2000 a 25.77 b 25.77 c 46.27
Dyn step 3000 a 25.77 b 25.77 c 52.27
Dyn step 4000 a 25.77 b 25.77 c 58.26
Dyn step 5000 a 25.77 b 25.77 c 64.26
Dyn step 6000 a 25.77 b 25.77 c 70.26
Dyn step 7000 a 25.77 b 25.77 c 76.26
Dyn step 8000 a 25.77 b 25.77 c 82.26
Dyn step 9000 a 25.77 b 25.77 c 88.26
Dyn step 10000 a 25.77 b 25.77 c 94.25

✅ 引張計算完了: PVDF_on_Al_cell_repeat_2x2x1
📊 結果サマリー:
  ヤング率: 0.

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_Al2O3_cell_repeat_2x2x1_P0.000_T300K_equil.traj.
Note: The max disk size of /home/jovyan is about 98G.
The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_Al2O3_cell_repeat_2x2x1_P0.000_T300K_HT1600K.traj.
Note: The max disk size of /home/jovyan is about 98G.


自由領域統一固定層設定:
  総材料厚さ: 45.56Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 23.26Å
  基板領域: 0.00 - 24.19Å
  PVDF領域: 22.33 - 45.56Å
  自由領域: 5.26 - 41.26Å
  実際の自由領域厚さ: 36.00Å
  固定原子数: 478/2502 (19.1%)
  自由原子数: 2024/2502 (80.9%)


In [ ]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
(PVDF-Al界面構造対応 修正版 V2)

主な修正点:
1. 入力ファイル形式をCIFに変更
2. PVDF-Al系のファイル命名規則に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
5. 圧着ステップ(Step1)を削除
6. ★ 2_build_interfaces_v2.py の新しいファイル命名規則
   (_repeat_..., _cropped_... など) に対応
"""

from ase.io import read, Trajectory, write
# ... (他の import は変更なし) ...
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
from ase.constraints import FixAtoms
import numpy as np
import re
import os

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# (変更なし)
# =============================================
# ... (BASE_DIR, INPUT_DIR などの定義) ...
# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure","output","PVDF_interfaces_onlyH2O")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDF")
# OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDFedit")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile01")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)


# =============================================
# 統合システム対応のファイルパス関数 (修正あり)
# =============================================

def get_interface_files():
    """
    統合システムで生成された界面構造ファイルを取得
    ★ 新旧の命名規則 (PVDF_on_... と PVDF_press_on_...) の両方を対象
    """
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.xyz')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出 (PVDF-Al系)
            # ★ 新しい形式 'PVDF_on_' と 古い形式 'PVDF_press_on_' の両方を許可
            if "PVDF_on_" in cif_file or "PVDF_press_on_" in cif_file:
                interface_files.append(cif_file)
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """
    界面構造ファイル名から材料情報を解析 (PVDF-Al系に対応)
    ★ 新しい命名規則（_repeat_... や _cropped_...）に対応 (V2)

    ファイル名例:
    - "PVDF_on_Al_metal_repeat_2x2x1_final.cif"
    - "PVDF_on_Al2O3_cropped_size_15x15_final.cif"
    - "PVDF_on_AlF3_original_cell_final.cif"
    - "PVDF_press_on_AlF3_final.cif" (古い形式もサポート)
    """
    
    # .cif を削除
    base_name_for_error = Path(filename).stem # "PVDF_on_Al_metal_repeat_2x2x1_final"
    
    # --- 1. "_final" を削除 ---
    if base_name_for_error.endswith("_final"):
        base_name_no_final = base_name_for_error[:-len("_final")]
    elif base_name_for_error.endswith("_optimization"):
        base_name_no_final = base_name_for_error[:-len("_optimization")]
    # elif base_name_for_error.endswith("_pre_opt"):
        # base_name_no_final = base_name_for_error[:-len("_optimization")]
    else:
        # 古い形式 (PVDF_press_on_...) は _final が無い場合もあるため、
        # "PVDF_press_on_" で始まらない場合は _final が必須だったとみなし、
        # 解析エラーになるよう "unknown" を返す
        if not base_name_for_error.startswith("PVDF_press_on_"):
             print(f"⏩ スキップ: ファイル名に '_final' が含まれていません: {filename}")
             return None
        
        # 古い形式 "PVDF_press_on_..." の場合は _final がなくても解析を試みる
        base_name_no_final = base_name_for_error

    # --- 2. 新しいサフィックス (_repeat_, _cropped_, _original_) を特定・分離 ---
    # (例: "PVDF_on_Al_metal_repeat_2x2x1")
    
    substrate_type_suffix = "unknown" # "repeat_2x2x1", "cropped_size_15x15", "original_cell" など
    core_name = base_name_no_final # "PVDF_on_Al_metal_repeat_2x2x1"
    
    # 正規表現でサフィックスを検索
    # (末尾の _repeat_... or _cropped_abs_... or _cropped_size_... or _original_cell)
    # (注: _cropped_abs_... は Lx_str が "20_5" のように "_" を含む可能性があるため、
    #    単純な \w+ ではなく .* (貪欲でない) を使う必要があるが、
    #    ここでは末尾までをサフィックスとみなすため、パターンを限定する)
    pattern = r"^(.*?)(_repeat_\d+x\d+x\d+|_cropped_abs_[\d_x]+|_cropped_size_[\d_x]+|_original_cell)$"
    match = re.match(pattern, base_name_no_final)
    
    if match:
        core_name = match.group(1) # "PVDF_on_Al_metal"
        substrate_type_suffix = match.group(2)[1:] # "_repeat_..." -> "repeat_..."
    else:
        # マッチしない場合 (古い形式 "PVDF_press_on_Al_metal" など)
        core_name = base_name_no_final
        substrate_type_suffix = "legacy_format" # 古い形式

    # --- 3. コア部分 ("PVDF_on_Al_metal" or "PVDF_press_on_Al_metal") を解析 ---
    
    # "PVDF_press_on_" (古い形式) または "PVDF_on_" (新しい形式) で分割
    parts = []
    if "_on_" in core_name:
        # 貪欲に分割 (例: "PVDF_press_on_Al_metal")
        parts = core_name.split('_on_', 1) 
    
    if len(parts) != 2:
        # 古い形式 "PVDF_press_on_..." を再チェック
        if core_name.startswith("PVDF_press_on_"):
            pvdf_part_str = "PVDF_press"
            al_part_str = core_name[len("PVDF_press_on_"):]
            if substrate_type_suffix == "legacy_format":
                substrate_type_suffix = "legacy_press_format" # 古い形式であることを明記
        else:
            print(f"⏩ スキップ: ファイル名のコア部分 '{core_name}' (from {filename}) が '_on_' で分割できません。")
            return None
    else:
        pvdf_part_str = parts[0] # "PVDF" or "PVDF_press"
        al_part_str = parts[1]   # "Al_metal", "Al2O3", "AlF3"
    
    # --- 4. 各コンポーネントを整理 ---
    
    # Al側（基板）の解析
    al_component = al_part_str # "Al_metal", "Al2O3", "AlF3"
    
    # PVDF側（上部）の解析
    nmc_component = "PVDF" # 便宜的にnmc_component変数名を利用
    nmc_variant = "unknown"
    
    if pvdf_part_str == "PVDF":
        nmc_component = "PVDF"
        nmc_variant = "standard" # 新しい形式
    elif pvdf_part_str == "PVDF_press":
        nmc_component = "PVDF"
        nmc_variant = "press" # 古い形式
    else:
        # 万が一 "Something_else_on_Al_metal" だった場合
        nmc_component = pvdf_part_str
        nmc_variant = "custom_top"
        
    # miller_al の代わりに substrate_type_suffix を使用
    # miller_nmc は "unknown" または "bulk" (古い形式の場合)
    miller_nmc = "unknown"
    if nmc_variant == "press":
        miller_nmc = "bulk"

    # base_name は {上部}_{基板}_{サフィックス} に変更
    # (例: "PVDF_on_Al_metal_repeat_2x2x1")
    # ただし、サフィックスが "legacy_format" や "legacy_press_format" の場合は追加しない
    base_name = f"{pvdf_part_str}_on_{al_component}"
    if substrate_type_suffix not in ["legacy_format", "legacy_press_format", "unknown"]:
        base_name += f"_{substrate_type_suffix}"
    
    # 古い形式 "PVDF_press_on_Al_metal" の場合、
    # substrate_type_suffix は "legacy_press_format" になり、
    # base_name は "PVDF_press_on_Al_metal" となる。
    
    # 新しい形式 "PVDF_on_Al_metal_repeat_2x2x1" の場合、
    # substrate_type_suffix は "repeat_2x2x1" になり、
    # base_name は "PVDF_on_Al_metal_repeat_2x2x1" となる。

    return {
        'al_component': al_component,      # 基板: "Al_metal", "Al2O3", "AlF3"
        'nmc_component': nmc_component,  # 上部: "PVDF"
        'nmc_variant': nmc_variant,      # 上部詳細: "standard" or "press"
        'miller_al': substrate_type_suffix, # 基板詳細: "repeat_2x2x1", "cropped_...", "original_cell", "legacy_..."
        'miller_nmc': miller_nmc,        # 上部詳細: "unknown" or "bulk"
        'base_name': base_name           # "PVDF_on_Al_metal_repeat_2x2x1" 等
    }

# ... (get_input_file_path から save_interface_results までの関数は変更なし) ...
def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

# 圧着ステップ削除に伴い、get_compression_file_path は削除

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}__P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,         # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,           # 界面構造用vacuum（Å）
    "temperature": 300.0,        # 引張温度固定（K）
    "max_strain": 1.75,          # 界面用最大ひずみ
    "timestep": 1.0,             # fs
    "elastic_region": (0.005, 0.03), # 界面用弾性領域
    "quality_threshold": 0.85,     # 界面用R²値閾値
    "max_steps": 10000,          # 界面用最大ステップ数
    "output_freq": 50,           # 出力頻度
    "free_region_thickness": 36.0  # 界面構造用固定層厚さ（Å）
}

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,         # 界面分離判定の隙間（Å）
    "max_strain_limit": 1.75,      # 最大ひずみ制限
    "min_stress_threshold": -0.5,  # 最小応力閾値（GPa）
    "separation_wait_steps": 300,  # 分離後の待機ステップ
    "stress_drop_ratio": 0.7       # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
# 圧着ステップは削除されたが、ファイル名命名規則のために変数は残す
compression_temperatures = [300]     # 圧着シミュレーション温度（K）
compression_pressures = [0.00]       # 圧着圧力（GPa）
high_temperatures = [1600]            # 高温シミュレーション温度（K）
tensile_temperature = 300.0          # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定 (PVDF-Al系に対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とPVDF原子（C, H, F）を分離
    # ★ Al2O3, AlF3 にも対応するため、Al 以外の原子も Al 層とみなす必要がある
    #    → PVDF (C, H, F) 以外の全てを "基板" とみなす
    
    pvdf_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['C', 'H', 'F']]
    # ★ 基板 (Substrate) は PVDF 以外の全原子
    substrate_indices = [i for i, atom in enumerate(atoms) if atom.symbol not in ['C', 'H', 'F']]
    
    # (元の al_indices を substrate_indices に変更)
    if not substrate_indices or not pvdf_indices:
        # 界面でない場合のフォールバック
        print("警告: identify_interface_region で基板またはPVDF原子が見つかりません。")
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)), # 'al_region' 変数名は維持
            'pvdf_region': (np.mean(z_coords), np.max(z_coords)), # 'nmc_region' -> 'pvdf_region'
            'is_interface': False
        }
    
    substrate_z_coords = z_coords[substrate_indices]
    pvdf_z_coords = z_coords[pvdf_indices]
    
    # Al領域とPVDF領域の境界を特定
    substrate_center = np.mean(substrate_z_coords) # al_center -> substrate_center
    pvdf_center = np.mean(pvdf_z_coords)
    
    if substrate_center < pvdf_center:
        # 基板が下、PVDFが上
        interface_z = (np.max(substrate_z_coords) + np.min(pvdf_z_coords)) / 2
        al_region = (np.min(substrate_z_coords), np.max(substrate_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    else:
        # PVDFが下、基板が上
        print("逆向きです。 (PVDFが下, 基板が上)")
        interface_z = (np.max(pvdf_z_coords) + np.min(substrate_z_coords)) / 2
        al_region = (np.min(substrate_z_coords), np.max(substrate_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,     # 'al_region' 変数名は維持 (基板領域)
        'pvdf_region': pvdf_region,   # 'nmc_region' -> 'pvdf_region'
        'al_center': substrate_center, # 'al_center' 変数名は維持 (基板中心)
        'pvdf_center': pvdf_center,   # 'nmc_center' -> 'pvdf_center'
        'is_interface': True
    }

def set_fixed_atoms_under(atoms,thickness=3.0):
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    indices = [i for i, z in enumerate(z_coords)if z < 3.0]
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    return atoms

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定 (PVDF-Al系対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得 (この関数は PVDF-Al 対応版)
    interface_info = identify_interface_region(atoms) 
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  基板領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å") # Al -> 基板
        print(f"  PVDF領域: {interface_info['pvdf_region'][0]:.2f} - {interface_info['pvdf_region'][1]:.2f}Å") # NMC -> PVDF
        
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)
        free_z_max = min(free_z_max, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 構造情報を属性として保存（固定原子がなくても厚さ情報は保存）
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)

    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        atoms.info['fixed_atoms_count'] = 0
        atoms.info['free_atoms_count'] = len(atoms)
        # 既存の制約がクリアされていることを確認
        atoms.set_constraint() 
        return atoms

    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms


# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器 (PVDF-Al系対応)"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_pvdf_separation = False # al_nmc_separation -> al_pvdf_separation
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出 (PVDF-Al系対応)"""
        
        # Al-PVDF界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms) # PVDF-Al対応版
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region'] # 基板領域
                pvdf_region = interface_info['pvdf_region'] # nmc_region -> pvdf_region
                
                # Al-PVDF間の隙間を計算
                if interface_info['al_center'] < interface_info['pvdf_center']: # nmc_center -> pvdf_center
                    gap = pvdf_region[0] - al_region[1] # nmc_region -> pvdf_region
                else:
                    gap = al_region[0] - pvdf_region[1] # nmc_region -> pvdf_region
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_pvdf_separation:
                    self.al_pvdf_separation = True # al_nmc_separation -> al_pvdf_separation
                    self.debonding_step = system.current_total_step
                    print(f"         🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] / units.GPa
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15: 
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"         ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_pvdf_separation: # al_nmc_separation -> al_pvdf_separation
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す (PVDF-Al系対応)"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_pvdf_separation, # al_nmc_separation -> al_pvdf_separation
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理 (PVDF-Al系対応)"""
    
    print("=" * 80)
    print("PVDF-Al 界面構造MD解析システム")
    print("統合界面構造生成システムとの連携版 (V2)")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    processed_count = 0
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            original_count = len(interface_files)
            if not interface_files:
                print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
                return
            # original_count = len(interface_files)
            interface_files = [
                f for f in interface_files 
                if not Path(f).stem.endswith("_pre_opt")
            ]
            print(f"解析対象の界面構造: {len(interface_files)}個")
            removed_count = original_count - len(interface_files)
            if material_info is None:
                continue # スキップ
            
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"  材料: {material_info}")
            print(f"  BaseName: {base_name}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # ファイルパス設定
                            # 圧着ステップは削除
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)

                            # ステップ1: 圧着 (削除)
                            # 圧着ステップは実行しない

                            # ステップ2: 平衡化 & 高温処理
                            if not check_md_completed(high_temp_file, 10000, 100):
                                print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                
                                # 圧着後ファイル(compression_file)の代わりに、
                                # 直接入力ファイル(input_file)を読む
                                atoms_after_comp = read(input_file)
                                
                                # ★ 元のスクリプトにあった固定層のロジック (V1 でコメントアウトされていたもの)
                                #    ここでは V2 の固定層設定 (set_interface_fixed_atoms_uniform_free)
                                #    を prepare_interface_structure_for_md の中で行うため、
                                #    ここでの固定層設定は不要 (V1 のロジックも削除)
                                
                                # pos=atoms_after_comp.get_positions()
                                # indices_to_delete1 = np.where((pos[:, 2] >= 100))
                                # del atoms_after_comp[indices_to_delete1]
                                # positions = atoms_after_comp.get_positions()
                                # z_coords = positions[:, 2]
                                # indices = [i for i, z in enumerate(z_coords)if z > 35 or z<5]
                                # if len(indices) == 0:
                                #     print("  警告: 固定原子がありません")
                                #     # return atoms # main() の中なので return は不適切
                                # else:
                                #     constraint = FixAtoms(indices=indices)
                                #     atoms_after_comp.set_constraint(constraint)
                                
                                # ★ V2: set_interface_fixed_atoms_uniform_free を使う
                                #     (ただし、高温処理の前に再度設定する)
                                atoms_after_comp = set_interface_fixed_atoms_uniform_free(
                                    atoms_after_comp, 
                                    UNIFIED_CONDITIONS["free_region_thickness"]
                                )
                                
                                # 平衡化 (NVT, 3ps @ 300K)
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(300.0)
                                integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                                
                                # 高温処理 (NVT, 10ps @ high_temp)
                                atoms_after_equil = read(equil_file, index='-1')
                                
                                # ★ 高温処理の前に固定層を再設定
                                atoms_after_equil = set_interface_fixed_atoms_uniform_free(
                                    atoms_after_equil, 
                                    UNIFIED_CONDITIONS["free_region_thickness"]
                                )
                                
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(float(high_temp))
                                integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                md = MDFeature(integrator, n_run=100000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                            else:
                                print("高温シミュレーションは既に完了しています。")

                            # ステップ3: 引張試験
                            if not os.path.exists(summary_file):
                                print("引張シミュレーション実行中...")
                                final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                
                                print(f"\n✅ 引張計算完了: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                            else:
                                print("引張シミュレーションと解析は既に完了しています。")

                        except Exception as e:
                            print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                            import traceback
                            traceback.print_exc()
                            continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        # ステップ数が期待値の90%以上あれば完了とみなす
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """
    界面構造ファイルを読み込み、MD計算用に準備する関数。
    (PVDF-Al系対応 / 修正済みバージョン)
    """
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # --- 1. 構造の読み込みと基本設定 ---
    atoms = Trajectory(input_file)[-1]
    atoms.set_constraint() # 既存の拘束をクリア

    #--- 2. 構造の向きを調整 (基板が下層になるように条件付きで反転) ---
    # (★ identify_interface_region を使うように変更)
    interface_info_check = identify_interface_region(atoms)
    
    if interface_info_check['is_interface']:
        # al_center は "基板中心" を意味する
        if interface_info_check['al_center'] > interface_info_check['pvdf_center']:
            print("構造チェック: 基板層が上部に検出されたため、構造をZ軸反転します。")
            positions = atoms.get_positions()
            positions[:, 2] *= -1
            atoms.set_positions(positions)
        else:
            print("構造チェック: 基板層は既に下部にあるため、構造の反転はスキップします。")
    else:
        print("構造チェック: 界面が検出できなかったため、反転処理をスキップします。")

    # --- 3. 構造情報の解析とz_minの0への移動 ---
    positions = atoms.get_positions()
    z_min = np.min(positions[:, 2])
    
    atoms.translate([0, 0, -z_min])
    print(f"構造移動: z_min ({z_min:.2f} Å) が 0 Å になるように構造全体を移動しました。")

    z_coords = atoms.get_positions()[:, 2]
    z_min_new, z_max_new = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max_new - z_min_new
    
    print("界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f} Å")
    print(f"  原子数: {len(atoms)}")

    # --- 4. セルと真空層の設定 ---
    # vacuum_top 引数を尊重せず、1.0 Å を追加する (元のコードの挙動を維持)
    vacuum_to_add = 1.0 
    print(f"設定: 構造上部に {vacuum_to_add} Å の真空層を追加します。")
    
    new_cell_z = material_thickness + vacuum_to_add
    cell = atoms.get_cell()
    cell[2, 2] = new_cell_z
    atoms.set_cell(cell, scale_atoms=False) 

    # ひずみ基準長
    strain_reference_length = material_thickness
    strain_method = "material_length"

    # --- 5. 自由領域・固定領域の設定 ---
    # この時点で atoms.info に 'free_region_thickness' などが設定される
    atoms = set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness) 

    # --- 6. 構造情報を `atoms.info` に集約 ---
    interface_info = identify_interface_region(atoms) # 情報を再取得
    initial_reference = atoms.get_cell()[2, 2]
    total_cell_z = atoms.get_cell()[2, 2]

    # atoms.info を更新 (set_interface_fixed... で追加された情報以外を update)
    atoms.info.update({
        "input_file": os.path.basename(input_file),
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": total_cell_z,
        "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
        "strain_reference_length": strain_reference_length,
        "strain_method": strain_method,
        "interface_info": interface_info,
        # 'free_region_thickness' などは set_interface... 関数内で既に atoms.info に追加されている
    })
    
    # 呼び出し元に返す structure_info 辞書を作成
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
        "interface_info": interface_info,
        "strain_method": strain_method,
        
        # set_interface_fixed... で設定された情報を atoms.info から取得
        "free_region_thickness": atoms.info.get('free_region_thickness', None),
        "free_region_bounds": atoms.info.get('free_region_bounds', None),
        "fixed_atoms_count": atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": atoms.info.get('free_atoms_count', len(atoms))
    }
    
    print("準備完了:")
    print(f"  最終セルZ長: {atoms.info['total_cell_z']:.2f} Å")
    print(f"  材料占有率: {atoms.info['material_ratio']*100:.1f}%")
    if atoms.info['interface_info'].get('is_interface', False):
        print(f"  界面検出: Yes (Z ≈ {atoms.info['interface_info'].get('interface_z', 0.0):.2f} Å)")
    else:
        print(f"  界面検出: No")
    
    return atoms, initial_reference, structure_info
    

# 圧着ステップ削除に伴い、run_interface_compression_md は削除

def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    ase_atoms,initial_reference,structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if initial_reference == 0:
        initial_reference = 50 # Fallback value to avoid division by zero
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")

    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版 / PVDF-Al系対応）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"PVDF-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  基板成分 (Al側): {material_info['al_component']}\n")
        f.write(f"  上部成分 (PVDF側): {material_info['nmc_component']}\n") # nmc_component 変数名だが中身は PVDF
        f.write(f"  上部組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  基板 Miller指数/タイプ: {material_info['miller_al']}\n") # ★ 名前変更
        f.write(f"  上部 Miller指数/タイプ: {material_info['miller_nmc']}\n") # ★ 名前変更
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        free_count = structure_info.get('free_atoms_count', 0)
        fixed_count = structure_info.get('fixed_atoms_count', 1) # Avoid division by zero
        total_atoms = free_count + fixed_count
        free_ratio = free_count / total_atoms if total_atoms > 0 else 0

        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {free_count}\n")
        f.write(f"  固定原子数: {fixed_count}\n")
        f.write(f"  自由原子比率: {free_ratio * 100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力 (ファイル名用): {pressure:.3f} GPa\n")
        f.write(f"  温度 (ファイル名用): {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
            
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")


if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
(PVDF-Al界面構造対応 修正版 V2)

主な修正点:
1. 入力ファイル形式をCIFに変更
2. PVDF-Al系のファイル命名規則に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
5. 圧着ステップ(Step1)を削除
6. ★ 2_build_interfaces_v2.py の新しいファイル命名規則
   (_repeat_..., _cropped_... など) に対応
"""

from ase.io import read, Trajectory, write
# ... (他の import は変更なし) ...
import re

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# (変更なし)
# =============================================
# ... (BASE_DIR, INPUT_DIR などの定義) ...
# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure","output","PVDF_interfaces")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDF")
# OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_PVDFedit")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile01")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)


# =============================================
# 統合システム対応のファイルパス関数 (修正あり)
# =============================================

def get_interface_files():
    """
    統合システムで生成された界面構造ファイルを取得
    ★ 新旧の命名規則 (PVDF_on_... と PVDF_press_on_...) の両方を対象
    """
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.xyz')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出 (PVDF-Al系)
            # ★ 新しい形式 'PVDF_on_' と 古い形式 'PVDF_press_on_' の両方を許可
            if "PVDF_on_" in cif_file or "PVDF_press_on_" in cif_file:
                interface_files.append(cif_file)
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """
    界面構造ファイル名から材料情報を解析 (PVDF-Al系に対応)
    ★ 新しい命名規則（_repeat_... や _cropped_...）に対応 (V2)

    ファイル名例:
    - "PVDF_on_Al_metal_repeat_2x2x1_final.cif"
    - "PVDF_on_Al2O3_cropped_size_15x15_final.cif"
    - "PVDF_on_AlF3_original_cell_final.cif"
    - "PVDF_press_on_AlF3_final.cif" (古い形式もサポート)
    """
    
    # .cif を削除
    base_name_for_error = Path(filename).stem # "PVDF_on_Al_metal_repeat_2x2x1_final"
    
    # --- 1. "_final" を削除 ---
    if base_name_for_error.endswith("_final"):
        base_name_no_final = base_name_for_error[:-len("_final")]
    elif base_name_for_error.endswith("_optimization"):
        base_name_no_final = base_name_for_error[:-len("_optimization")]
    # elif base_name_for_error.endswith("_pre_opt"):
        # base_name_no_final = base_name_for_error[:-len("_optimization")]
    else:
        # 古い形式 (PVDF_press_on_...) は _final が無い場合もあるため、
        # "PVDF_press_on_" で始まらない場合は _final が必須だったとみなし、
        # 解析エラーになるよう "unknown" を返す
        if not base_name_for_error.startswith("PVDF_press_on_"):
             print(f"⏩ スキップ: ファイル名に '_final' が含まれていません: {filename}")
             return None
        
        # 古い形式 "PVDF_press_on_..." の場合は _final がなくても解析を試みる
        base_name_no_final = base_name_for_error

    # --- 2. 新しいサフィックス (_repeat_, _cropped_, _original_) を特定・分離 ---
    # (例: "PVDF_on_Al_metal_repeat_2x2x1")
    
    substrate_type_suffix = "unknown" # "repeat_2x2x1", "cropped_size_15x15", "original_cell" など
    core_name = base_name_no_final # "PVDF_on_Al_metal_repeat_2x2x1"
    
    # 正規表現でサフィックスを検索
    # (末尾の _repeat_... or _cropped_abs_... or _cropped_size_... or _original_cell)
    # (注: _cropped_abs_... は Lx_str が "20_5" のように "_" を含む可能性があるため、
    #    単純な \w+ ではなく .* (貪欲でない) を使う必要があるが、
    #    ここでは末尾までをサフィックスとみなすため、パターンを限定する)
    pattern = r"^(.*?)(_repeat_\d+x\d+x\d+|_cropped_abs_[\d_x]+|_cropped_size_[\d_x]+|_original_cell)$"
    match = re.match(pattern, base_name_no_final)
    
    if match:
        core_name = match.group(1) # "PVDF_on_Al_metal"
        substrate_type_suffix = match.group(2)[1:] # "_repeat_..." -> "repeat_..."
    else:
        # マッチしない場合 (古い形式 "PVDF_press_on_Al_metal" など)
        core_name = base_name_no_final
        substrate_type_suffix = "legacy_format" # 古い形式

    # --- 3. コア部分 ("PVDF_on_Al_metal" or "PVDF_press_on_Al_metal") を解析 ---
    
    # "PVDF_press_on_" (古い形式) または "PVDF_on_" (新しい形式) で分割
    parts = []
    if "_on_" in core_name:
        # 貪欲に分割 (例: "PVDF_press_on_Al_metal")
        parts = core_name.split('_on_', 1) 
    
    if len(parts) != 2:
        # 古い形式 "PVDF_press_on_..." を再チェック
        if core_name.startswith("PVDF_press_on_"):
            pvdf_part_str = "PVDF_press"
            al_part_str = core_name[len("PVDF_press_on_"):]
            if substrate_type_suffix == "legacy_format":
                substrate_type_suffix = "legacy_press_format" # 古い形式であることを明記
        else:
            print(f"⏩ スキップ: ファイル名のコア部分 '{core_name}' (from {filename}) が '_on_' で分割できません。")
            return None
    else:
        pvdf_part_str = parts[0] # "PVDF" or "PVDF_press"
        al_part_str = parts[1]   # "Al_metal", "Al2O3", "AlF3"
    
    # --- 4. 各コンポーネントを整理 ---
    
    # Al側（基板）の解析
    al_component = al_part_str # "Al_metal", "Al2O3", "AlF3"
    
    # PVDF側（上部）の解析
    nmc_component = "PVDF" # 便宜的にnmc_component変数名を利用
    nmc_variant = "unknown"
    
    if pvdf_part_str == "PVDF":
        nmc_component = "PVDF"
        nmc_variant = "standard" # 新しい形式
    elif pvdf_part_str == "PVDF_press":
        nmc_component = "PVDF"
        nmc_variant = "press" # 古い形式
    else:
        # 万が一 "Something_else_on_Al_metal" だった場合
        nmc_component = pvdf_part_str
        nmc_variant = "custom_top"
        
    # miller_al の代わりに substrate_type_suffix を使用
    # miller_nmc は "unknown" または "bulk" (古い形式の場合)
    miller_nmc = "unknown"
    if nmc_variant == "press":
        miller_nmc = "bulk"

    # base_name は {上部}_{基板}_{サフィックス} に変更
    # (例: "PVDF_on_Al_metal_repeat_2x2x1")
    # ただし、サフィックスが "legacy_format" や "legacy_press_format" の場合は追加しない
    base_name = f"{pvdf_part_str}_on_{al_component}"
    if substrate_type_suffix not in ["legacy_format", "legacy_press_format", "unknown"]:
        base_name += f"_{substrate_type_suffix}"
    
    # 古い形式 "PVDF_press_on_Al_metal" の場合、
    # substrate_type_suffix は "legacy_press_format" になり、
    # base_name は "PVDF_press_on_Al_metal" となる。
    
    # 新しい形式 "PVDF_on_Al_metal_repeat_2x2x1" の場合、
    # substrate_type_suffix は "repeat_2x2x1" になり、
    # base_name は "PVDF_on_Al_metal_repeat_2x2x1" となる。

    return {
        'al_component': al_component,      # 基板: "Al_metal", "Al2O3", "AlF3"
        'nmc_component': nmc_component,  # 上部: "PVDF"
        'nmc_variant': nmc_variant,      # 上部詳細: "standard" or "press"
        'miller_al': substrate_type_suffix, # 基板詳細: "repeat_2x2x1", "cropped_...", "original_cell", "legacy_..."
        'miller_nmc': miller_nmc,        # 上部詳細: "unknown" or "bulk"
        'base_name': base_name           # "PVDF_on_Al_metal_repeat_2x2x1" 等
    }

# ... (get_input_file_path から save_interface_results までの関数は変更なし) ...
def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

# 圧着ステップ削除に伴い、get_compression_file_path は削除

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}__P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,         # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,           # 界面構造用vacuum（Å）
    "temperature": 300.0,        # 引張温度固定（K）
    "max_strain": 1.75,          # 界面用最大ひずみ
    "timestep": 1.0,             # fs
    "elastic_region": (0.005, 0.03), # 界面用弾性領域
    "quality_threshold": 0.85,     # 界面用R²値閾値
    "max_steps": 10000,          # 界面用最大ステップ数
    "output_freq": 50,           # 出力頻度
    "free_region_thickness": 36.0  # 界面構造用固定層厚さ（Å）
}

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,         # 界面分離判定の隙間（Å）
    "max_strain_limit": 1.75,      # 最大ひずみ制限
    "min_stress_threshold": -0.5,  # 最小応力閾値（GPa）
    "separation_wait_steps": 300,  # 分離後の待機ステップ
    "stress_drop_ratio": 0.7       # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
# 圧着ステップは削除されたが、ファイル名命名規則のために変数は残す
compression_temperatures = [300]     # 圧着シミュレーション温度（K）
compression_pressures = [0.00]       # 圧着圧力（GPa）
high_temperatures = [500]            # 高温シミュレーション温度（K）
tensile_temperature = 300.0          # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定 (PVDF-Al系に対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とPVDF原子（C, H, F）を分離
    # ★ Al2O3, AlF3 にも対応するため、Al 以外の原子も Al 層とみなす必要がある
    #    → PVDF (C, H, F) 以外の全てを "基板" とみなす
    
    pvdf_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['C', 'H', 'F']]
    # ★ 基板 (Substrate) は PVDF 以外の全原子
    substrate_indices = [i for i, atom in enumerate(atoms) if atom.symbol not in ['C', 'H', 'F']]
    
    # (元の al_indices を substrate_indices に変更)
    if not substrate_indices or not pvdf_indices:
        # 界面でない場合のフォールバック
        print("警告: identify_interface_region で基板またはPVDF原子が見つかりません。")
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)), # 'al_region' 変数名は維持
            'pvdf_region': (np.mean(z_coords), np.max(z_coords)), # 'nmc_region' -> 'pvdf_region'
            'is_interface': False
        }
    
    substrate_z_coords = z_coords[substrate_indices]
    pvdf_z_coords = z_coords[pvdf_indices]
    
    # Al領域とPVDF領域の境界を特定
    substrate_center = np.mean(substrate_z_coords) # al_center -> substrate_center
    pvdf_center = np.mean(pvdf_z_coords)
    
    if substrate_center < pvdf_center:
        # 基板が下、PVDFが上
        interface_z = (np.max(substrate_z_coords) + np.min(pvdf_z_coords)) / 2
        al_region = (np.min(substrate_z_coords), np.max(substrate_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    else:
        # PVDFが下、基板が上
        print("逆向きです。 (PVDFが下, 基板が上)")
        interface_z = (np.max(pvdf_z_coords) + np.min(substrate_z_coords)) / 2
        al_region = (np.min(substrate_z_coords), np.max(substrate_z_coords))
        pvdf_region = (np.min(pvdf_z_coords), np.max(pvdf_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,     # 'al_region' 変数名は維持 (基板領域)
        'pvdf_region': pvdf_region,   # 'nmc_region' -> 'pvdf_region'
        'al_center': substrate_center, # 'al_center' 変数名は維持 (基板中心)
        'pvdf_center': pvdf_center,   # 'nmc_center' -> 'pvdf_center'
        'is_interface': True
    }

def set_fixed_atoms_under(atoms,thickness=3.0):
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    indices = [i for i, z in enumerate(z_coords)if z < 3.0]
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    return atoms

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定 (PVDF-Al系対応)"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得 (この関数は PVDF-Al 対応版)
    interface_info = identify_interface_region(atoms) 
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  基板領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å") # Al -> 基板
        print(f"  PVDF領域: {interface_info['pvdf_region'][0]:.2f} - {interface_info['pvdf_region'][1]:.2f}Å") # NMC -> PVDF
        
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)
        free_z_max = min(free_z_max, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 構造情報を属性として保存（固定原子がなくても厚さ情報は保存）
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)

    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        atoms.info['fixed_atoms_count'] = 0
        atoms.info['free_atoms_count'] = len(atoms)
        # 既存の制約がクリアされていることを確認
        atoms.set_constraint() 
        return atoms

    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms


# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器 (PVDF-Al系対応)"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_pvdf_separation = False # al_nmc_separation -> al_pvdf_separation
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出 (PVDF-Al系対応)"""
        
        # Al-PVDF界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms) # PVDF-Al対応版
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region'] # 基板領域
                pvdf_region = interface_info['pvdf_region'] # nmc_region -> pvdf_region
                
                # Al-PVDF間の隙間を計算
                if interface_info['al_center'] < interface_info['pvdf_center']: # nmc_center -> pvdf_center
                    gap = pvdf_region[0] - al_region[1] # nmc_region -> pvdf_region
                else:
                    gap = al_region[0] - pvdf_region[1] # nmc_region -> pvdf_region
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_pvdf_separation:
                    self.al_pvdf_separation = True # al_nmc_separation -> al_pvdf_separation
                    self.debonding_step = system.current_total_step
                    print(f"         🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] / units.GPa
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15: 
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"         ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_pvdf_separation: # al_nmc_separation -> al_pvdf_separation
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す (PVDF-Al系対応)"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_pvdf_separation, # al_nmc_separation -> al_pvdf_separation
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理 (PVDF-Al系対応)"""
    
    print("=" * 80)
    print("PVDF-Al 界面構造MD解析システム")
    print("統合界面構造生成システムとの連携版 (V2)")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    processed_count = 0
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            original_count = len(interface_files)
            if not interface_files:
                print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
                return
            # original_count = len(interface_files)
            interface_files = [
                f for f in interface_files 
                if not Path(f).stem.endswith("_pre_opt")
            ]
            print(f"解析対象の界面構造: {len(interface_files)}個")
            removed_count = original_count - len(interface_files)
            if material_info is None:
                continue # スキップ
            
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"  材料: {material_info}")
            print(f"  BaseName: {base_name}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # ファイルパス設定
                            # 圧着ステップは削除
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)

                            # ステップ1: 圧着 (削除)
                            # 圧着ステップは実行しない

                            # ステップ2: 平衡化 & 高温処理
                            if not check_md_completed(high_temp_file, 10000, 100):
                                print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                
                                # 圧着後ファイル(compression_file)の代わりに、
                                # 直接入力ファイル(input_file)を読む
                                atoms_after_comp = read(input_file)
                                
                                # ★ 元のスクリプトにあった固定層のロジック (V1 でコメントアウトされていたもの)
                                #    ここでは V2 の固定層設定 (set_interface_fixed_atoms_uniform_free)
                                #    を prepare_interface_structure_for_md の中で行うため、
                                #    ここでの固定層設定は不要 (V1 のロジックも削除)
                                
                                # pos=atoms_after_comp.get_positions()
                                # indices_to_delete1 = np.where((pos[:, 2] >= 100))
                                # del atoms_after_comp[indices_to_delete1]
                                # positions = atoms_after_comp.get_positions()
                                # z_coords = positions[:, 2]
                                # indices = [i for i, z in enumerate(z_coords)if z > 35 or z<5]
                                # if len(indices) == 0:
                                #     print("  警告: 固定原子がありません")
                                #     # return atoms # main() の中なので return は不適切
                                # else:
                                #     constraint = FixAtoms(indices=indices)
                                #     atoms_after_comp.set_constraint(constraint)
                                
                                # ★ V2: set_interface_fixed_atoms_uniform_free を使う
                                #     (ただし、高温処理の前に再度設定する)
                                atoms_after_comp = set_interface_fixed_atoms_uniform_free(
                                    atoms_after_comp, 
                                    UNIFIED_CONDITIONS["free_region_thickness"]
                                )
                                
                                # 平衡化 (NVT, 3ps @ 300K)
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(300.0)
                                integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                                
                                # 高温処理 (NVT, 10ps @ high_temp)
                                atoms_after_equil = read(equil_file, index='-1')
                                
                                # ★ 高温処理の前に固定層を再設定
                                atoms_after_equil = set_interface_fixed_atoms_uniform_free(
                                    atoms_after_equil, 
                                    UNIFIED_CONDITIONS["free_region_thickness"]
                                )
                                
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(float(high_temp))
                                integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                            else:
                                print("高温シミュレーションは既に完了しています。")

                            # ステップ3: 引張試験
                            if not os.path.exists(summary_file):
                                print("引張シミュレーション実行中...")
                                final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                
                                print(f"\n✅ 引張計算完了: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                            else:
                                print("引張シミュレーションと解析は既に完了しています。")

                        except Exception as e:
                            print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                            import traceback
                            traceback.print_exc()
                            continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        # ステップ数が期待値の90%以上あれば完了とみなす
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """
    界面構造ファイルを読み込み、MD計算用に準備する関数。
    (PVDF-Al系対応 / 修正済みバージョン)
    """
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # --- 1. 構造の読み込みと基本設定 ---
    atoms = Trajectory(input_file)[-1]
    atoms.set_constraint() # 既存の拘束をクリア

    #--- 2. 構造の向きを調整 (基板が下層になるように条件付きで反転) ---
    # (★ identify_interface_region を使うように変更)
    interface_info_check = identify_interface_region(atoms)
    
    if interface_info_check['is_interface']:
        # al_center は "基板中心" を意味する
        if interface_info_check['al_center'] > interface_info_check['pvdf_center']:
            print("構造チェック: 基板層が上部に検出されたため、構造をZ軸反転します。")
            positions = atoms.get_positions()
            positions[:, 2] *= -1
            atoms.set_positions(positions)
        else:
            print("構造チェック: 基板層は既に下部にあるため、構造の反転はスキップします。")
    else:
        print("構造チェック: 界面が検出できなかったため、反転処理をスキップします。")

    # --- 3. 構造情報の解析とz_minの0への移動 ---
    positions = atoms.get_positions()
    z_min = np.min(positions[:, 2])
    
    atoms.translate([0, 0, -z_min])
    print(f"構造移動: z_min ({z_min:.2f} Å) が 0 Å になるように構造全体を移動しました。")

    z_coords = atoms.get_positions()[:, 2]
    z_min_new, z_max_new = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max_new - z_min_new
    
    print("界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f} Å")
    print(f"  原子数: {len(atoms)}")

    # --- 4. セルと真空層の設定 ---
    # vacuum_top 引数を尊重せず、1.0 Å を追加する (元のコードの挙動を維持)
    vacuum_to_add = 1.0 
    print(f"設定: 構造上部に {vacuum_to_add} Å の真空層を追加します。")
    
    new_cell_z = material_thickness + vacuum_to_add
    cell = atoms.get_cell()
    cell[2, 2] = new_cell_z
    atoms.set_cell(cell, scale_atoms=False) 

    # ひずみ基準長
    strain_reference_length = material_thickness
    strain_method = "material_length"

    # --- 5. 自由領域・固定領域の設定 ---
    # この時点で atoms.info に 'free_region_thickness' などが設定される
    atoms = set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness) 

    # --- 6. 構造情報を `atoms.info` に集約 ---
    interface_info = identify_interface_region(atoms) # 情報を再取得
    initial_reference = atoms.get_cell()[2, 2]
    total_cell_z = atoms.get_cell()[2, 2]

    # atoms.info を更新 (set_interface_fixed... で追加された情報以外を update)
    atoms.info.update({
        "input_file": os.path.basename(input_file),
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": total_cell_z,
        "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
        "strain_reference_length": strain_reference_length,
        "strain_method": strain_method,
        "interface_info": interface_info,
        # 'free_region_thickness' などは set_interface... 関数内で既に atoms.info に追加されている
    })
    
    # 呼び出し元に返す structure_info 辞書を作成
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
        "interface_info": interface_info,
        "strain_method": strain_method,
        
        # set_interface_fixed... で設定された情報を atoms.info から取得
        "free_region_thickness": atoms.info.get('free_region_thickness', None),
        "free_region_bounds": atoms.info.get('free_region_bounds', None),
        "fixed_atoms_count": atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": atoms.info.get('free_atoms_count', len(atoms))
    }
    
    print("準備完了:")
    print(f"  最終セルZ長: {atoms.info['total_cell_z']:.2f} Å")
    print(f"  材料占有率: {atoms.info['material_ratio']*100:.1f}%")
    if atoms.info['interface_info'].get('is_interface', False):
        print(f"  界面検出: Yes (Z ≈ {atoms.info['interface_info'].get('interface_z', 0.0):.2f} Å)")
    else:
        print(f"  界面検出: No")
    
    return atoms, initial_reference, structure_info
    

# 圧着ステップ削除に伴い、run_interface_compression_md は削除

def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    ase_atoms,initial_reference,structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if initial_reference == 0:
        initial_reference = 50 # Fallback value to avoid division by zero
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")

    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版 / PVDF-Al系対応）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"PVDF-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  基板成分 (Al側): {material_info['al_component']}\n")
        f.write(f"  上部成分 (PVDF側): {material_info['nmc_component']}\n") # nmc_component 変数名だが中身は PVDF
        f.write(f"  上部組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  基板 Miller指数/タイプ: {material_info['miller_al']}\n") # ★ 名前変更
        f.write(f"  上部 Miller指数/タイプ: {material_info['miller_nmc']}\n") # ★ 名前変更
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        free_count = structure_info.get('free_atoms_count', 0)
        fixed_count = structure_info.get('fixed_atoms_count', 1) # Avoid division by zero
        total_atoms = free_count + fixed_count
        free_ratio = free_count / total_atoms if total_atoms > 0 else 0

        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {free_count}\n")
        f.write(f"  固定原子数: {fixed_count}\n")
        f.write(f"  自由原子比率: {free_ratio * 100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力 (ファイル名用): {pressure:.3f} GPa\n")
        f.write(f"  温度 (ファイル名用): {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
            
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")


if __name__ == "__main__":
    main()

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_AlF3_cell_repeat_1x1x2_P0.000_T300K_equil.traj.
Note: The max disk size of /home/jovyan is about 98G.


PVDF-Al 界面構造MD解析システム
統合界面構造生成システムとの連携版 (V2)
発見された界面構造ファイル数: 35
解析対象の界面構造: 35個
⏩ スキップ: ファイル名に '_final' が含まれていません: PVDF_on_Al2O3_cell_repeat_1x1x3_pre_opt.xyz
解析対象の界面構造: 31個
⏩ スキップ: ファイル名に '_final' が含まれていません: PVDF_on_Al_cell_original_cell_with_H2O_d1_0_initial.xyz
解析対象の界面構造: 31個
⏩ スキップ: ファイル名に '_final' が含まれていません: PVDF_on_Al2O3_cell_original_cell_with_H2O_d1_0_initial.xyz
解析対象の界面構造: 31個
⏩ スキップ: ファイル名に '_final' が含まれていません: PVDF_on_Al2O3_cell_repeat_2x2x1_initial.xyz
解析対象の界面構造: 31個
⏩ スキップ: ファイル名に '_final' が含まれていません: PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_initial.xyz
解析対象の界面構造: 31個
解析対象の界面構造: 31個

処理開始: PVDF_on_AlF3_cell_repeat_1x1x2_final.xyz
  材料: {'al_component': 'AlF3_cell', 'nmc_component': 'PVDF', 'nmc_variant': 'standard', 'miller_al': 'repeat_1x1x2', 'miller_nmc': 'unknown', 'base_name': 'PVDF_on_AlF3_cell_repeat_1x1x2'}
  BaseName: PVDF_on_AlF3_cell_repeat_1x1x2

--- 条件: P0.000_T300K_HT500K ---
平衡化と高温シミュレーション(500K)を実行中...
自由領域統一固定層設定:
  総材料厚さ: 74.21Å
  目標自由領域厚さ: 36.00Å
  界面構造検

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_AlF3_cell_repeat_1x1x2_P0.000_T300K_HT500K.traj.
Note: The max disk size of /home/jovyan is about 98G.


自由領域統一固定層設定:
  総材料厚さ: 74.21Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 18.53Å
  基板領域: 0.00 - 37.06Å
  PVDF領域: 0.00 - 74.21Å
  自由領域: 2.00 - 36.53Å
  実際の自由領域厚さ: 34.53Å
  固定原子数: 464/928 (50.0%)
  自由原子数: 464/928 (50.0%)


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/tensile01/PVDF_on_AlF3_cell_repeat_1x1x2_P0.000_T300K_HT500K_tensile300K.traj.
Note: The max disk size of /home/jovyan is about 98G.


...

Dyn step 10000 a 12.89 b 12.89 c 92.53

✅ 引張計算完了: PVDF_on_Al_cell_original_cell_with_H2O_d1_0
📊 結果サマリー:
  ヤング率: 0.0 GPa (R² = 0.000)
  最大応力: 4.98 GPa
解析対象の界面構造: 31個

処理開始: PVDF_on_Al2O3_cell_repeat_1x1x3_final.xyz
  材料: {'al_component': 'Al2O3_cell', 'nmc_component': 'PVDF', 'nmc_variant': 'standard', 'miller_al': 'repeat_1x1x3', 'miller_nmc': 'unknown', 'base_name': 'PVDF_on_Al2O3_cell_repeat_1x1x3'}
  BaseName: PVDF_on_Al2O3_cell_repeat_1x1x3

--- 条件: P0.000_T300K_HT500K ---
平衡化と高温シミュレーション(500K)を実行中...
自由領域統一固定層設定:
  総材料厚さ: 120.61Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 62.76Å
  基板領域: 0.00 - 62.41Å
  PVDF領域: 63.10 - 120.61Å
  自由領域: 44.76 - 80.76Å
  実際の自由領域厚さ: 36.00Å
  固定原子数: 1293/1858 (69.6%)
  自由原子数: 565/1858 (30.4%)


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_Al2O3_cell_repeat_1x1x3_P0.000_T300K_HT500K.traj.
Note: The max disk size of /home/jovyan is about 98G.


自由領域統一固定層設定:
  総材料厚さ: 120.61Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 63.03Å
  基板領域: 0.00 - 63.00Å
  PVDF領域: 63.06 - 120.61Å
  自由領域: 45.03 - 81.03Å
  実際の自由領域厚さ: 36.00Å
  固定原子数: 1289/1858 (69.4%)
  自由原子数: 569/1858 (30.6%)


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/tensile01/PVDF_on_Al2O3_cell_repeat_1x1x3_P0.000_T300K_HT500K_tensile300K.traj.
Note: The max disk size of /home/jovyan is about 98G.


引張シミュレーション実行中...
構造チェック: 基板層は既に下部にあるため、構造の反転はスキップします。
構造移動: z_min (0.00 Å) が 0 Å になるように構造全体を移動しました。
界面構造解析:
  材料厚さ: 120.61 Å
  原子数: 1858
設定: 構造上部に 1.0 Å の真空層を追加します。
自由領域統一固定層設定:
  総材料厚さ: 120.61Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 63.12Å
  基板領域: 0.00 - 63.13Å
  PVDF領域: 63.11 - 120.61Å
  自由領域: 45.12 - 81.12Å
  実際の自由領域厚さ: 36.00Å
  固定原子数: 1292/1858 (69.5%)
  自由原子数: 566/1858 (30.5%)
準備完了:
  最終セルZ長: 121.61 Å
  材料占有率: 99.2%
  界面検出: Yes (Z ≈ 63.12 Å)
自由領域統一引張設定:
  統一自由領域厚さ: 36.00Å
  自由原子数: 566
  固定原子数: 1292
  引張ひずみ速度: 0.01 1/ps
Dyn step    0 a 13.56 b 11.63 c 121.61
